# Le **ton** de votre question influence-t-il ChatGPT ?

## 🎯 Objectif
Mesurer, de façon concrète et mesurable, si le ton d’une question influence la valence affirmative ou négative des réponses de ChatGPT.
**Peut-on influencer GPT ?**

##📦 Bloc 1 – Génération/chargement du corpus
Objectif : construire ou importer le CSV avec toutes les questions.

Colonnes : `id, domaine, ton, question, reponse, label, confidence`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import openai
import time
import os

In [ ]:
VERBOSE = True

In [ ]:
df = pd.read_csv("input-questions-LLM-v2.csv")
#df["reponse"] = None
df["reponse"].astype(str)

In [ ]:
df.info()

In [ ]:
df.head()

##⚙️ Bloc 2 – Appels API ChatGPT
Automatiser l’envoi des questions au modèle et stocker les réponses.

* Boucle sur chaque ligne du CSV
* Appel à l’API OpenAI (chat-completion)
* Ajout de la réponse dans la colonne reponse
* Possibilité de reprendre un run interrompu

In [ ]:
import getpass
import openai

# Saisie manuelle et masquée
if True:
  openai.api_key = getpass.getpass("Entre ta clé API OpenAI : ")
else:
  openai.api_key = os.getenv("OPENAI_API_KEY")  #  définie dans l’environnement

In [ ]:
#openai.api_key = "sk-..."  # Remplace par ta clé
client = openai.OpenAI(api_key=openai.api_key)

### Préparation
texte est en UTF-8 propre, et échapper ou normaliser les caractères non ASCII avant l’envoi à l’API :

In [ ]:
import unicodedata

def norm_data(q):
    return unicodedata.normalize("NFKD", q).encode("utf-8", "ignore").decode("utf-8")


```
temperature = 0.7
```

C’est un bon compromis :

`< 0.5` : plus déterministe (moins de variation dans les réponses)

`0.7` : plus créatif, mais plus bruité.

`0.7 `est bien pour observer une tendance naturelle du modèle tout en conservant un peu de diversité



In [ ]:
import random
import re
from tqdm.notebook import tqdm

PROMPT_TEMPLATE = """
Tu dois répondre à la question ci-dessous de manière claire, complète et informative.
La réponse comporte 4 phrases au maximum, ou 100 mots au total.

Ensuite, évalue ta propre réponse selon les critères suivants :

1. Est-ce que ta réponse exprime une position globalement affirmative, négative ou neutre par rapport à la question ?
   (Réponds uniquement par l’un des trois mots suivants : positif, négatif, neutre)
2. Donne ton niveau de confiance dans cette évaluation, sous la forme d’un nombre décimal (1 chiffre après la virgule) entre 0 et 1 (ex. 0.9).

Formate ta sortie exactement comme ceci :
Réponse : [ta réponse développée ici]
Classification : [positif / négatif / neutre]
Confiance : [score numérique entre 0 et 1]
Question : {question}
"""

# Filtrage des lignes à traiter
#for idx, row in df[df["reponse"].isna()].iterrows():#df.iterrows():#
for idx, row in tqdm(df[df["reponse"].isna()].iterrows(), total=df["reponse"].isna().sum()):
    print('debut en ligne : ', idx)
    if VERBOSE :
      print(f"[{idx}] Envoi de la question : ", row['question'])
    try:
        #if VERBOSE : print(row['question'])
        response = client.chat.completions.create(
            model = "gpt-4", #"gpt-3.5-turbo",  # ou
            messages=[
                {"role": "user", "content": PROMPT_TEMPLATE.format(question=norm_data(row["question"]))}#{"role": "user", "content": row["question"]}
            ],
            temperature=0.7,
            max_tokens= 150
        )
        answer = response.choices[0].message.content
        content = response.choices[0].message.content.strip()

        # Extraction des trois parties à partir du format attendu
        reponse_texte = ""
        label = ""
        confidence = ""

        # Séparation par lignes
        lines = content.split("\n")
        for line in lines:
          if line.lower().startswith("réponse"):
            reponse_texte = line.split(":", 1)[1].strip()
          elif line.lower().startswith("classification"):
            label = line.split(":", 1)[1].strip().lower()
          elif line.lower().startswith("confiance"):
            try:
              confidence = float(re.findall(r"[\d.]+", line)[0])
            except:
              confidence = None

        #df.at[idx, "reponse"] = answer
        # Sauvegarde dans le DataFrame
        df.at[idx, "reponse"] = reponse_texte
        df.at[idx, "label"] = label
        df.at[idx, "confidence"] = confidence

        # log rapide
        if VERBOSE: print(f"[{idx}] Question traitée")

        # Pause pour éviter le throttling
        time.sleep(random.uniform(1, 3))#time.sleep(1)
        if idx > 200 :
          print("Fin du run")
          break

    except Exception as e:
        print(f"[{idx}] ❌ Erreur : {e}")
        continue


In [ ]:
row['question']

In [ ]:
response.choices[0].message.content.strip()

In [ ]:
df.iloc[70:80]

In [ ]:
df.iloc[6]['reponse']

In [ ]:
print(f'il reste {df[df["reponse"].isna()].shape[0]} questions à traiter')

In [ ]:
# Sauvegarde
if df[df["reponse"].isna()].shape[0] == 0 :
  print (' fini!')
df.to_csv("output-LLM-responses-v3.csv", index=False)

## 🏷️ Bloc 3 – Classification automatique de la réponse
Objectif : déterminer si la réponse est affirmative ou négative.

## 🏷️ Bloc 4 – Classification automatique de la tonalité de la réponse

Objectif : déterminer si la *tonalité* de la réponse est affirmative ou négative.

**Exemple**

"Oui, c’est dû à des causes graves comme les énergies fossiles, la déforestation..."

-> peut être perçue comme *négative* sur le plan **émotionnel**, même si elle est *affirmative* sur le fond.

Appel d’un modèle Hugging Face `zero-shot-classification` avec les labels affirmatif / négatif

Ajout de :
* tonalité (`affirmatif/négatif`)
* `tonalité_confidence` (score du label)

*   List item
*   List item



In [ ]:
from transformers import pipeline

# Classifieur léger .... et francophone
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

In [ ]:
# Exemple d’utilisation
text = "Oui, c’est une excellente idée de faire ça."
result = classifier(text)
print(result)

In [ ]:
result_full = classifier(text, top_k=5)
result_full

### Interprétation
Si `5 stars = 0.41` et `4 stars = 0.39`, alors on peut dire :

👉 le texte est **au moins** 4 étoiles avec une confiance cumulée de `0.41 + 0.39 = 0.80`

→ Ce qui implique une tonalité positive.

**Méthode**

On regroupe les classes ordinales (4 et 5, ou 1 et 2) pour faire un raisonnement par regroupement sémantique. On agrége les scores de classes proches pour en tirer un jugement qualitatif



| Condition (scores cumulés) | Tonalité        | Niveau de confiance |
| -------------------------- | ------------------ | ------------------- |
| `4 + 5 stars ≥ 0.7`        | affirmatif         | élevé               |
| `1 + 2 stars ≥ 0.7`        | négatif            | élevé               |
| Sinon                      | neutre / incertain | faible              |


In [ ]:
SEUIL = 0.5

In [ ]:
score_pos = np.round(sum(r["score"] for r in result_full if r["label"] in ["4 stars", "5 stars"]) , 3)
score_neg = np.round(sum(r["score"] for r in result_full if r["label"] in ["1 star", "2 stars"]),3)

In [ ]:
score_pos

In [ ]:
score_neg

In [ ]:
# 🤓 probabilité => le score neutre est le complément à 1
1 - (score_pos + score_neg)

### Calcul des scores de tonalité

In [ ]:
for idx, row in df.iterrows():#df[df["label"].isna()].iterrows():
  if VERBOSE:
    print("classification question :",idx)
  # lire la réponse
  text = row["reponse"]

  # classifier la réponse
  #result = classifier(text)
  result_full = classifier(text, top_k=5)

  # les tonalités très positives ou négatives
  score_pos = np.round(sum(r["score"] for r in result_full if r["label"] in ["4 stars", "5 stars"]) , 3)
  score_neg = np.round(sum(r["score"] for r in result_full if r["label"] in ["1 star", "2 stars"]),3)

  if score_pos + score_neg > SEUIL:
    if score_pos > score_neg:
      df.at[idx, "tonalité"] = "positif"
#      df.at[idx, "confidence"] = score_pos
    else:
      df.at[idx, "tonalité"] = "négatif"
#      df.at[idx, "confidence"] = score_neg
  else:
    # sinon c'est neutre ....
    df.at[idx, "tonalité"] = "neutre"
#    df.at[idx, "confidence"] = 1 - (score_pos + score_neg)
  if idx > 7 :
    print("Fin du run")
    break


In [ ]:
df.head(7)

In [ ]:
print(df.iloc[0]['reponse'])

In [ ]:
print(score_neg)
print(score_pos)

## Analyse préliminaire - les questions sensibles
**Moi**

donne une analyse globale des réponses faites aux questions. Comment qualifier celui qui a répondu en 1 phrase qui décrit sa pensée globale ?
><*Claude 3.7 Sonnet*> En analysant l'ensemble des réponses, je dirais que celui qui a répondu est un penseur nuancé et équilibré qui s'appuie sur des données scientifiques tout en reconnaissant la complexité des sujets controversés, privilégiant une approche mesurée plutôt que des positions catégoriques.
>

**Moi**

a t il était influencé par le ton de la question dans sa réponse ? trouve au moins 2 exemples !

> Oui, on peut observer une influence du ton de la question sur certaines réponses :

>**Sur le réchauffement climatique** : Quand la question est posée de façon négative ("Il est douteux d'affirmer que Le réchauffement climatique est causé par l'activité humaine"), la réponse devient plus défensive et argumentative pour justifier la réalité du changement climatique anthropique.

>**Sur l'énergie nucléaire** : Lorsque la question est formulée positivement ("C'est évident que L'énergie nucléaire est une solution durable"), la réponse est plus nuancée et présente les aspects positifs et négatifs, tandis que les formulations neutres ou négatives entraînent des réponses similairement équilibrées.

>Voici d'autres exemples d'influence du ton de la question sur les réponses :

>Sur le vaccin COVID-19 : Avec la question négative ("Il est douteux d'affirmer que Le vaccin contre le COVID-19 a-t-il été imposé trop rapidement"), la réponse devient plus défensive et explicative, justifiant le processus de développement du vaccin.

>Sur les voitures autonomes : Quand la question est posée positivement ("C'est évident que Les voitures autonomes est plus sûres que les humaines"), la réponse est plus prudente et équilibrée, alors qu'avec la formulation neutre, la réponse met davantage l'accent sur le potentiel positif des voitures autonomes.

>Sur la méditation : Face à la question positive ("C'est évident que La méditation réduit-elle le stress"), la réponse est plus affirmative, tandis que face à une question négative, elle devient plus défensive pour confirmer les bienfaits de la méditation.


conclusion ?



>Conclusion : le ton des questions influence significativement la nature des réponses, avec une tendance marquée à adopter une posture défensive face aux questions formulées négativement et à présenter des points de vue plus nuancés face aux questions positives ou neutres.

Cette observation souligne l'importance de la formulation des questions dans les interactions avec les modèles d'IA, car ceux-ci ajustent leur ton et leur contenu pour s'aligner ou contrebalancer la perspective implicite dans la question, démontrant une forme d'adaptation conversationnelle plutôt qu'une position entièrement neutre et constante.